In [0]:
dbutils.fs.ls('/mnt/bronze/premData')

[FileInfo(path='dbfs:/mnt/bronze/premData/data_10d81911-a69f-4433-a15a-b6e1d4c20a2d_e996d02e-6f33-4b23-a45b-e1256509034f.parquet', name='data_10d81911-a69f-4433-a15a-b6e1d4c20a2d_e996d02e-6f33-4b23-a45b-e1256509034f.parquet', size=1064, modificationTime=1703671193000),
 FileInfo(path='dbfs:/mnt/bronze/premData/data_b9f74cd3-7766-43a5-8e10-156fa4e45da7_33491638-45f7-4214-800f-389216c18b2b.parquet', name='data_b9f74cd3-7766-43a5-8e10-156fa4e45da7_33491638-45f7-4214-800f-389216c18b2b.parquet', size=1650, modificationTime=1703671193000)]

In [0]:
dbutils.fs.ls('/mnt/silver')

[]

In [0]:
input_path = '/mnt/bronze/premData/data_10d81911-a69f-4433-a15a-b6e1d4c20a2d_e996d02e-6f33-4b23-a45b-e1256509034f.parquet'

In [0]:
df = spark.read.format('parquet').load(input_path)

In [0]:
display(df)

user_id,username,email,birthdate,registration_date
1,JohnDoe,john.doe@example.com,1990-05-15T00:00:00Z,2023-12-27T12:35:07Z
2,JaneSmith,jane.smith@example.com,1985-08-22T00:00:00Z,2023-12-27T12:35:07Z
3,BobJohnson,bob.johnson@example.com,1995-02-10T00:00:00Z,2023-12-27T12:35:07Z


In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType


df = df.withColumn("birthdate", date_format(from_utc_timestamp(df["birthdate"].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))
df = df.withColumn("registration_date", date_format(from_utc_timestamp(df["registration_date"].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))


In [0]:
display(df)

user_id,username,email,birthdate,registration_date
1,JohnDoe,john.doe@example.com,1990-05-15,2023-12-27
2,JaneSmith,jane.smith@example.com,1985-08-22,2023-12-27
3,BobJohnson,bob.johnson@example.com,1995-02-10,2023-12-27


Transformation for other table

In [0]:
table_name=[]
for i in dbutils.fs.ls('/mnt/bronze/premData'):
    table_name.append(i.name.split('/')[0])

In [0]:
table_name

['data_10d81911-a69f-4433-a15a-b6e1d4c20a2d_e996d02e-6f33-4b23-a45b-e1256509034f.parquet',
 'data_b9f74cd3-7766-43a5-8e10-156fa4e45da7_33491638-45f7-4214-800f-389216c18b2b.parquet']

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

for i in table_name:
    path = '/mnt/bronze/premData' + i + '/' + '.parquet'
    column =df.columns
    
    for col in column:
        if "Date" in col or "date" in col:
            df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))
    print(f"Transformed DataFrame for table {i}:")
    df.show()
    output_path = '/mnt/silver/premData/' +i +'/'
    df.write.format('delta').mode("overwrite").save(output_path)

Transformed DataFrame for table data_10d81911-a69f-4433-a15a-b6e1d4c20a2d_e996d02e-6f33-4b23-a45b-e1256509034f.parquet:
+-------+----------+--------------------+----------+-----------------+
|user_id|  username|               email| birthdate|registration_date|
+-------+----------+--------------------+----------+-----------------+
|      1|   JohnDoe|john.doe@example.com|2023-12-27|       2023-12-27|
|      2| JaneSmith|jane.smith@exampl...|2023-12-27|       2023-12-27|
|      3|BobJohnson|bob.johnson@examp...|2023-12-27|       2023-12-27|
+-------+----------+--------------------+----------+-----------------+

Transformed DataFrame for table data_b9f74cd3-7766-43a5-8e10-156fa4e45da7_33491638-45f7-4214-800f-389216c18b2b.parquet:
+-------+----------+--------------------+----------+-----------------+
|user_id|  username|               email| birthdate|registration_date|
+-------+----------+--------------------+----------+-----------------+
|      1|   JohnDoe|john.doe@example.com|2023-12-

In [0]:
display(table_name[1])

'data_b9f74cd3-7766-43a5-8e10-156fa4e45da7_33491638-45f7-4214-800f-389216c18b2b.parquet'